In [ ]:
# 📦 Step 0: Extract mushrooms.zip (again if needed)
import zipfile

zip_path = '/content/Mushrooms(1).zip'
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall()
print("✅ Extracted into 'Mushrooms/'")


✅ Extracted into 'Mushrooms/'


In [ ]:
# 🧹 Step 1: Clean truncated images
import os
from PIL import Image, UnidentifiedImageError, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

corrupted_files = []
root_dir = 'Mushrooms'

for subdir, _, files in os.walk(root_dir):
    for file in files:
        file_path = os.path.join(subdir, file)
        try:
            with Image.open(file_path) as img:
                img.verify()
        except (UnidentifiedImageError, IOError, OSError):
            corrupted_files.append(file_path)

for f in corrupted_files:
    os.remove(f)

print(f"✅ Cleaned {len(corrupted_files)} corrupted images")


✅ Cleaned 1 corrupted images


In [ ]:
# 📥 Step 2: Load data for feature extraction
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pickle

IMAGE_SIZE = (128, 128)
BATCH_SIZE = 1

datagen = ImageDataGenerator(rescale=1./255)
generator = datagen.flow_from_directory(
    'Mushrooms',
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='sparse',
    shuffle=False
)

# Save class index mapping
with open("class_indices_svm.pkl", "wb") as f:
    pickle.dump(generator.class_indices, f)


Found 6714 images belonging to 9 classes.


In [ ]:
# 🧠 Step 3: Build feature extractor model
from tensorflow.keras import layers, models

def build_feature_extractor():
    model = models.Sequential([
        layers.Input(shape=(*IMAGE_SIZE, 3)),
        layers.Conv2D(32, 3, activation='relu'),
        layers.MaxPooling2D(),
        layers.Conv2D(64, 3, activation='relu'),
        layers.MaxPooling2D(),
        layers.Conv2D(128, 3, activation='relu'),
        layers.MaxPooling2D(),
        layers.Flatten(),
        layers.Dense(128, activation='relu')
    ])
    return model

extractor = build_feature_extractor()


In [ ]:
# 🔍 Step 4: Extract features and labels
features = []
labels = []

for i in range(len(generator)):
    x, y = generator[i]
    feat = extractor.predict(x)
    features.append(feat[0])
    labels.append(int(y[0]))

features = np.array(features)
labels = np.array(labels)
print("✅ Feature shape:", features.shape)


Streaming output truncated to the last 5000 lines.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step

In [ ]:
# 🤖 Step 5: Train and save SVM
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

svm = SVC(kernel='rbf')
svm.fit(features, labels)

# Save models
extractor.save('feature_extractor_model.keras')

with open('svm_model.pkl', 'wb') as f:
    pickle.dump(svm, f)

print("✅ Feature extractor and SVM saved.")


✅ Feature extractor and SVM saved.


In [ ]:
from sklearn.metrics import accuracy_score

# Predict on training data
preds = svm.predict(features)

# Compare to actual labels
acc = accuracy_score(labels, preds)


print(f"✅ SVM Training Accuracy: {acc * 100:.2f}%")


✅ SVM Training Accuracy: 34.72%
